In [1]:
import os
import copy

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

from comet_ml import Experiment

from models.MBNV3withCBAM import MobileNetV3CBAM

import config

from models.functions import train, evaluate, confusion



/Users/leo/Desktop/programming/bp_classifier/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_data = 'images'
classes = sorted(os.listdir(training_data))
print(classes)

['.DS_Store', 'albino', 'banana', 'butter', 'clown', 'ghi', 'hypo', 'lesser', 'mojave', 'piebald', 'spider']


In [3]:
experiment = Experiment(project_name = "BP_MobileNet", workspace = "leothesouthafrican", api_key = config.api_key)

hyper_params = {
    'learning_rate': 0.001,
    'num_epochs': 30,
    'batch_size': 32,
    'image_size': 192,
    'image_channels': 3,
    'output_size': len(classes),
    'num_layers': 'na',
    'train_val_split': 0.90,
    'device': 'mps',
    'model_name': 'Basic MobileNetV3',
    'criterion': 'CrossEntropyLoss',
    'optimizer': 'Adam',
    'dataset': 'CIFAR10',
    'best_model_path': 'MN3Small_bp.pt',
}

#Setting the device
device = torch.device(hyper_params['device'])

# Loading model
model = MobileNetV3CBAM(mode='large')
model.to(device)

# Setting the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device) #Setting the loss function
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate']) #Setting the optimizer

# Adding model parameters to comet
for name, param in model.named_parameters():
    hyper_params[name] = param

# Logging the hyperparameters to comet
experiment.log_parameters(hyper_params)

COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/leothesouthafrican/bp-mobilenet/ad50636519f746028c661e6a2c57f9ef



In [4]:
def load_dataset(data_path, batch_size=32, shuffle=True):
    #load dataset
    train_transform = transforms.Compose([
                                        transforms.Resize((hyper_params['image_size'], hyper_params['image_size'])),
                                        transforms.RandomHorizontalFlip(0.2),
                                        transforms.RandomVerticalFlip(0.2),
                                        transforms.RandomRotation(10),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    test_transform = transforms.Compose([
                                        transforms.Resize((hyper_params['image_size'], hyper_params['image_size'])),
                                        transforms.ToTensor()])

    full_dataset = torchvision.datasets.ImageFolder(root=data_path, transform=train_transform)
    print('Total Images in Dataset: {}'.format(len(full_dataset)))
    print('Classes: {}'.format(full_dataset.classes))
    print('--' * 15)
    #split dataset into training and validation
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
    print('Number of training images: {}'.format(len(train_dataset)))
    print('Number of test images: {}'.format(len(test_dataset)))

    #create dataloaders
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=hyper_params["batch_size"],
                                                shuffle=True,
                                                num_workers=8)

    test_loader = torch.utils.data.DataLoader(test_dataset,
                                                batch_size=hyper_params["batch_size"],
                                                shuffle=True,
                                                num_workers=8)

    n_train = int(len(train_dataset) * hyper_params['train_val_split'])
    n_val = len(train_dataset) - n_train

    train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [n_train, n_val])

    val_data = copy.deepcopy(train_dataset)
    val_data.dataset.transform = test_transform

    val_loader = torch.utils.data.DataLoader(val_data,
                                                batch_size=hyper_params["batch_size"],
                                                shuffle=False,
                                                num_workers=8)

    print('Number of validation images: {}'.format(len(val_dataset)))
    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = load_dataset(training_data)

Total Images in Dataset: 5598
Classes: ['albino', 'banana', 'butter', 'clown', 'ghi', 'hypo', 'lesser', 'mojave', 'piebald', 'spider']
------------------------------
Number of training images: 4478
Number of test images: 1120
Number of validation images: 448


In [5]:
with experiment.train():
    train(hyper_params['num_epochs'], model, criterion, optimizer, train_loader, val_loader, hyper_params['best_model_path'], device, experiment)

Begin training...


100%|██████████| 126/126 [00:54<00:00,  2.29it/s]


	Train Loss: 2.186 | Train Acc: 16.67%
Epoch: 02 | Epoch Time: 3m 4s
	 Val. Loss: 2.253 |  Val. Acc: 0.16%


100%|██████████| 126/126 [00:54<00:00,  2.31it/s]


	Train Loss: 2.291 | Train Acc: 16.67%
Epoch: 03 | Epoch Time: 2m 39s
	 Val. Loss: 2.297 |  Val. Acc: 0.11%


...

100%|██████████| 126/126 [00:54<00:00,  2.31it/s]


	Train Loss: 2.295 | Train Acc: 10.00%
Epoch: 10 | Epoch Time: 2m 41s
	 Val. Loss: 2.298 |  Val. Acc: 0.11%


100%|██████████| 126/126 [00:54<00:00,  2.31it/s]


	Train Loss: 2.287 | Train Acc: 3.33%
Epoch: 11 | Epoch Time: 2m 39s
	 Val. Loss: 2.294 |  Val. Acc: 0.11%


  1%|          | 1/140 [00:06<14:03,  6.07s/it]

In [ ]:
with experiment.test():

    model.load_state_dict(torch.load(hyper_params['best_model_path']))

    test_loss, test_acc = evaluate(model, test_loader, criterion, device, experiment)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
confusion(model, test_loader, device = device, experiment=experiment)

In [ ]:
experiment.end()